In [5]:
import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from obsq.utils.core import convert_df_to_gdf
path = "/home/etienne/projects/obsq/work/dev/data/data.duckdb"
target = 'expert_match'

pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [6]:
table = 'combined'
schema = 'features'
con = duckdb.connect(path)
df = con.execute(f"""SELECT* FROM {schema}.{table}""" ).df()
#df = df.drop(columns= 'geom')
con.close()

In [7]:
def plot_mean_response(df, x):
    df.groupby(x)[target].mean().plot(kind="bar")
    plt.ylabel("P(y=1 | x)")
    plt.xlabel("x")
    plt.show()

In [8]:
def plot_cat_rate(df, cat_col):
    output = "home/etienne/projects/obsq/eda"
    rates = (
        df.groupby(cat_col)[target]
        .mean()
        .sort_values()
    )
    sns.barplot(x=rates.index[:20], y=rates.values[:20])
    plt.xticks(rotation=45, ha='right')   # or rotation=60, ha='right'
    plt.title(f"Observations rates for {cat_col}", fontsize=16)
    plt.savefig(f"{output}/{cat_col}_target_rates.jpg", dpi = 300, bbox_inches="tight")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39886 entries, 0 to 39885
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   gbifID                           39886 non-null  int64  
 1   expert_match                     39886 non-null  int32  
 2   observations_count               39886 non-null  int64  
 3   total_pct                        39886 non-null  float64
 4   expert_ids                       39886 non-null  int64  
 5   expert_ids_pct                   39886 non-null  float64
 6   expert_match_count               39886 non-null  int64  
 7   expert_match_pct                 39886 non-null  float64
 8   avg_id_time                      39886 non-null  float64
 9   class_count                      39886 non-null  int64  
 10  order_count                      39886 non-null  int64  
 11  family_count                     39886 non-null  int64  
 12  genus_count       

In [10]:
df.describe()

,gbifID,expert_match,observations_count,total_pct,expert_ids,expert_ids_pct,expert_match_count,expert_match_pct,avg_id_time,class_count,...,expert_id,day,month,year,media_count,coordinateUncertaintyInMeters,pheno_01,pheno_02,pheno_03,occurrenceRemarks
count,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,...,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000,39886.00000
mean,4652105352.86988,0.06616,542.32360,0.01364,15.08650,0.04740,0.71965,0.01413,24.78067,3.84807,...,0.04728,15.86266,6.25124,2023.49265,17.26746,101.57770,0.16169,0.10227,0.00910,4.56303
std,717500937.25705,0.24857,949.69838,0.02389,23.44223,0.08595,0.44918,0.05408,48.78851,1.21310,...,0.21225,9.15706,1.25091,1.42943,12.73118,339.94187,0.36817,0.30300,0.09496,33.75805
min,3079585653.00000,0.00000,1.00000,0.00003,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,...,0.00000,1.00000,4.00000,2021.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,4102946510.25000,0.00000,32.00000,0.00080,0.00000,0.00000,0.00000,0.00000,2.00000,3.00000,...,0.00000,7.00000,5.00000,2022.00000,10.00000,2.00000,0.00000,0.00000,0.00000,0.00000
50%,4902078740.00000,0.00000,143.00000,0.00360,3.00000,0.01700,1.00000,0.00190,11.00000,4.00000,...,0.00000,16.00000,6.00000,2024.00000,10.00000,6.00000,0.00000,0.00000,0.00000,0.00000
75%,5187908635.25000,0.00000,491.00000,0.01235,19.00000,0.05600,1.00000,0.00740,21.00000,5.00000,...,0.00000,24.00000,7.00000,2025.00000,20.00000,30.00000,0.00000,0.00000,0.00000,0.00000
max,5897886507.00000,1.00000,3389.00000,0.08525,88.00000,1.00000,1.00000,1.00000,1455.00000,5.00000,...,1.00000,31.00000,8.00000,2025.00000,240.00000,2998.00000,1.00000,1.00000,1.00000,2126.00000


In [13]:
df.var(numeric_only = True).sort_values(ascending=False)


gbifID                            514807594964748608.00000
taxon_observations_total_count               1688404.32748
observations_count                            901927.00655
max_yearly_observations                       289248.46950
taxon_observation_month_count                 120252.31270
coordinateUncertaintyInMeters                 115560.47363
avg_yearly_observations                        54729.41658
avg_coord_un                                   42389.40899
max_monthly_observations                       37726.52539
species_count                                  13900.43122
id_time                                        11435.25388
genus_count                                     6525.32604
avg_monthly_observations                        5995.58230
avg_description_len                             5355.80580
unique_dates                                    4233.30331
max_species_obs_count                           3024.65914
avg_id_time                                     2380.318

In [ ]:
cols = 4
rows = 5
setup = {}
columns = df.select_dtypes(include='number').columns.tolist()

size = 4
f, axs = plt.subplots(rows,cols, figsize=(cols*size, rows*size), squeeze=False)
for i in range(0,rows):
    for j in range(0,cols):
        #print(i,j, (i*cols + j))
        index = i*cols + j
        if index > (cols*rows):
            break
        if columns[index] in ['latitude', 'longitude']:
            continue
        sns.histplot(df, hue = target, x = columns[index], ax=axs[i,j])


plt.show()

/home/etienne/miniconda3/envs/obsq/lib/python3.11/site-packages/IPython/core/pylabtools.py:170: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


KeyboardInterrupt: 

In [ ]:
for c in columns:
    print(round(df[c].std() / df[c].mean(),3), " ", c)

0.154   gbifID
3.757   expert_match
1.751   observations_count
1.751   total_pct
1.554   expert_ids
1.813   expert_ids_pct
0.624   expert_match_count
3.826   expert_match_pct
1.969   avg_id_time
0.315   class_count
0.502   order_count
0.666   family_count
0.83   genus_count
0.91   species_count
0.521   unique_year_count
1.172   unique_dates
1.704   max_yearly_observations
1.45   max_monthly_observations
1.572   avg_yearly_observations
1.518   avg_monthly_observations
1.667   high_cood_un_obs
3.297   high_cood_un_pct
2.028   avg_coord_un
0.49   avg_media_count
5.311   sex_meta_pct
2.065   reproductiveCondition_meta_pct
3.069   annotations_meta_pct
1.893   avg_description_len
2.239   avg_species_obs_count
2.038   max_species_obs_count
2.554   taxon_observations_total_count
2.799   taxon_observation_month_count
1.523   taxon_observation_month_density
0.423   id_count
0.018   id_agree_rate
4.323   id_time
4.489   expert_id
0.577   day
0.2   month
0.001   year
0.737   media_count
3.347   co

In [ ]:
# Factorize area id 
#codes, uniques = pd.factorize(df['source'])
#df['area_id'] = codes
#df = df.drop(columns=['source'])